In [1]:
import os
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
import pandas as pd
import pprint
import datetime as dt

In [2]:
url_corona = "https://www.worldometers.info/coronavirus/#countries"
html_corona = requests.get(url_corona)
html_corona = html_corona.text
#html_corona

In [3]:
soup_corona = bs(html_corona, 'html.parser')

In [4]:
corona_table = soup_corona.find(class_="table-bordered")

corona_table = str(corona_table)


In [5]:
corona_df = pd.read_html(corona_table)[0]

#replacing blanks with 0
corona_df.fillna(0,inplace = True)

#removing the total row)
corona_df = corona_df[~corona_df['Country,Other'].isin(['Total:', 'Total', 'total'])]
corona_df.to_csv('new_data.csv')
#replace items using wild card and regex to have that applied
corona_df["NewCases"].replace("\,","",regex = True, inplace = True)
corona_df["NewCases"].replace("\+","", regex = True, inplace = True)
corona_df["NewCases"] = corona_df["NewCases"].astype("float")
corona_df["NewDeaths"].replace("\,","",regex = True, inplace = True)
corona_df["NewDeaths"].replace("\+","", regex = True, inplace = True)
corona_df["NewDeaths"] = corona_df["NewDeaths"].astype("float")


#add date column for date data scrped
corona_df["Date"]= dt.datetime.today()

In [6]:
corona_df

,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/ 1M pop,Date
0,World,4224845,46691.0,285147.0,1413.0,1510980.0,2428718,47062.0,542.0,36.6,0.0,0.0,2020-05-11 12:37:02.305348
1,USA,1372740,5102.0,80933.0,146.0,256972.0,1034835,16514.0,4147.0,245.0,9518765.0,28757.0,2020-05-11 12:37:02.305348
2,Spain,268143,3480.0,26744.0,123.0,177846.0,63553,1650.0,5735.0,572.0,2467761.0,52781.0,2020-05-11 12:37:02.305348
3,UK,223060,3877.0,32065.0,210.0,0.0,190651,1559.0,3286.0,472.0,1921770.0,28309.0,2020-05-11 12:37:02.305348
4,Russia,221344,11656.0,2009.0,94.0,39801.0,179534,2300.0,1517.0,14.0,5636763.0,38625.0,2020-05-11 12:37:02.305348
...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,St. Barth,6,0.0,0.0,0.0,6.0,0,0.0,607.0,0.0,0.0,0.0,2020-05-11 12:37:02.305348
211,Western Sahara,6,0.0,0.0,0.0,5.0,1,0.0,10.0,0.0,0.0,0.0,2020-05-11 12:37:02.305348
212,Anguilla,3,0.0,0.0,0.0,3.0,0,0.0,200.0,0.0,0.0,0.0,2020-05-11 12:37:02.305348
213,Saint Pierre Miquelon,1,0.0,0.0,0.0,1.0,0,0.0,173.0,0.0,0.0,0.0,2020-05-11 12:37:02.305348


In [7]:
# adding lat long info 
countries_df = pd.read_csv("country_lan_lng.csv",engine='python')
merged = corona_df.merge(countries_df, left_on='Country,Other', right_on='name')
df_used = merged.drop(countries_df.columns[0:-2], axis=1)

df_used.rename(columns={"Country,Other":"Country","Serious,Critical": "CriticalCases"}, inplace = True)
#replace items using wild card and regex to have that applied
df_used["NewCases"].replace("\,","",regex = True, inplace = True)
df_used["NewCases"].replace("\+","", regex = True, inplace = True)
df_used["ClosedCases"] = df_used["TotalDeaths"] +  df_used["TotalRecovered"]

df_used["ClosedDeathRate"] = df_used["TotalDeaths"]/df_used["ClosedCases"]
df_used["ClosedDeathRate"].fillna(0,inplace = True)

df_used["CurrentDeathRate"] = df_used["TotalDeaths"]/df_used["TotalCases"]
df_used["CurrentDeathRate"].fillna(0,inplace = True)

total_cases = df_used["TotalCases"].sum()
total_closed_cases = df_used["ClosedCases"].sum()

df_used["Weight"] = [x/total_closed_cases for x in df_used["ClosedCases"]]

# #checking if total weight is 1 
# validate_weight = df_used["Weight"].sum()


df_used["DeathRateXWeight"] = df_used["CurrentDeathRate"] *df_used["Weight"]
df_used["DeathRateXWeight"].sum()
df_used["notes"] = ["Last 8 columns are added by Roaya" for x in df_used["DeathRateXWeight"]  ]
df_used.sort_values(by = "DeathRateXWeight", ascending = False)

,Country,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,ActiveCases,CriticalCases,Tot Cases/1M pop,Deaths/1M pop,...,Tests/ 1M pop,Date,Longitude,Latitude,ClosedCases,ClosedDeathRate,CurrentDeathRate,Weight,DeathRateXWeight,notes
1,Spain,268143,3480.0,26744.0,123.0,177846.0,63553,1650.0,5735.0,572.0,...,52781.0,2020-05-11 12:37:02.305348,-3.647550,40.244487,204590.0,0.130720,0.099738,0.114202,0.011390,Last 8 columns are added by Roaya
0,USA,1372740,5102.0,80933.0,146.0,256972.0,1034835,16514.0,4147.0,245.0,...,28757.0,2020-05-11 12:37:02.305348,-97.922211,39.381266,337905.0,0.239514,0.058957,0.188618,0.011120,Last 8 columns are added by Roaya
4,Italy,219814,744.0,30739.0,179.0,106587.0,82488,999.0,3636.0,508.0,...,43112.0,2020-05-11 12:37:02.305348,12.070013,42.796626,137326.0,0.223840,0.139841,0.076655,0.010720,Last 8 columns are added by Roaya
5,France,176970,0.0,26380.0,0.0,56217.0,94373,2776.0,2711.0,404.0,...,21213.0,2020-05-11 12:37:02.305348,2.618787,47.824905,82597.0,0.319382,0.149065,0.046105,0.006873,Last 8 columns are added by Roaya
6,Germany,171999,120.0,7569.0,0.0,145600.0,18830,1581.0,2053.0,90.0,...,32891.0,2020-05-11 12:37:02.305348,10.385781,51.106982,153169.0,0.049416,0.044006,0.085498,0.003762,Last 8 columns are added by Roaya
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,Mozambique,103,12.0,0.0,0.0,34.0,69,0.0,3.0,0.0,...,134.0,2020-05-11 12:37:02.305348,35.533675,-17.273816,34.0,0.000000,0.000000,0.000019,0.000000,Last 8 columns are added by Roaya
154,Macao,45,0.0,0.0,0.0,40.0,5,1.0,69.0,0.0,...,0.0,2020-05-11 12:37:02.305348,113.509321,22.223117,40.0,0.000000,0.000000,0.000022,0.000000,Last 8 columns are added by Roaya
160,Grenada,21,0.0,0.0,0.0,13.0,8,4.0,187.0,0.0,...,12495.0,2020-05-11 12:37:02.305348,-61.682202,12.117250,13.0,0.000000,0.000000,0.000007,0.000000,Last 8 columns are added by Roaya
159,Timor-Leste,24,0.0,0.0,0.0,21.0,3,0.0,18.0,0.0,...,560.0,2020-05-11 12:37:02.305348,125.844390,-8.828892,21.0,0.000000,0.000000,0.000012,0.000000,Last 8 columns are added by Roaya


In [8]:
df_used.to_csv('data.csv')
df_used.to_html('data.html')

In [9]:
corona_json = df_used.to_json(orient='records')
df_used.to_json('data.json',orient='records')

In [10]:
corona_json

'[{"Country":"USA","TotalCases":1372740,"NewCases":5102.0,"TotalDeaths":80933.0,"NewDeaths":146.0,"TotalRecovered":256972.0,"ActiveCases":1034835,"CriticalCases":16514.0,"Tot\\u00a0Cases\\/1M pop":4147.0,"Deaths\\/1M pop":245.0,"TotalTests":9518765.0,"Tests\\/ 1M pop":28757.0,"Date":1589200622305,"Longitude":-97.922211,"Latitude":39.381266,"ClosedCases":337905.0,"ClosedDeathRate":0.2395140646,"CurrentDeathRate":0.0589572679,"Weight":0.1886175803,"DeathRateXWeight":0.0111203772,"notes":"Last 8 columns are added by Roaya"},{"Country":"Spain","TotalCases":268143,"NewCases":3480.0,"TotalDeaths":26744.0,"NewDeaths":123.0,"TotalRecovered":177846.0,"ActiveCases":63553,"CriticalCases":1650.0,"Tot\\u00a0Cases\\/1M pop":5735.0,"Deaths\\/1M pop":572.0,"TotalTests":2467761.0,"Tests\\/ 1M pop":52781.0,"Date":1589200622305,"Longitude":-3.64755047,"Latitude":40.24448698,"ClosedCases":204590.0,"ClosedDeathRate":0.1307199765,"CurrentDeathRate":0.0997378265,"Weight":0.1142015382,"DeathRateXWeight":0.011